In [19]:
!pip install openpyxl
!pip install xlrd
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Excelファイルからデータを読み込む
# 'your_data.xlsx'を実際のファイル名に置き換えてください
df = pd.read_csv('C:/Users/thisi/OneDrive/デスクトップ/Dateset1.csv')

print("データの最初の5行:")
print(df.head(100))

print("\nデータの情報:")
print(df.info())

# 説明変数と目的変数を分離
# 'target_column'を目的変数の列名に置き換えてください
X = df.drop('CRIM', axis=1)
y = df['CRIM']

print("\n説明変数の列:")
print(X.columns)

# パイプラインの定義（正規化とSVR）
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# ハイパーパラメータの範囲を定義
param_grid = {
    'svr__C': [0.1, 1, 10, 100],
    'svr__epsilon': [0.01, 0.1, 0.5],
    'svr__kernel': ['rbf', 'linear'],
    'svr__gamma': ['scale', 'auto', 0.1, 1]
}

# 交差検証の設定
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCVの設定
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)

# グリッドサーチの実行
grid_search.fit(X, y)

# 最適なパラメータと最良のスコアを表示
print("\n最適なパラメータ:", grid_search.best_params_)
print("最良の負のMSE:", grid_search.best_score_)

# 最適なモデルを取得
best_model = grid_search.best_estimator_

# 交差検証を使用して最終的な性能を評価
cv_scores = cross_val_score(best_model, X, y, cv=cv, scoring='neg_mean_squared_error')
mse_scores = -cv_scores  # 負のMSEを正のMSEに変換
r2_scores = cross_val_score(best_model, X, y, cv=cv, scoring='r2')

print(f"\n交差検証MSE: {mse_scores.mean():.4f} (+/- {mse_scores.std() * 2:.4f})")
print(f"交差検証R-squared: {r2_scores.mean():.4f} (+/- {r2_scores.std() * 2:.4f})")

# 全データセットでモデルを再学習
best_model.fit(X, y)

# 予測を行い、最終的なスコアを計算
y_pred = best_model.predict(X)
final_mse = mean_squared_error(y, y_pred)
final_r2 = r2_score(y, y_pred)

print(f"\n全データセットでの最終MSE: {final_mse:.4f}")
print(f"全データセットでの最終R-squared: {final_r2:.4f}")

データの最初の5行:
       CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  PTRATIO  \
0   0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1   0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2   0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3   0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4   0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   
..      ...   ...    ...   ...    ...    ...   ...     ...  ...  ...      ...   
95  0.12204   0.0   2.89     0  0.445  6.625  57.8  3.4952    2  276     18.0   
96  0.11504   0.0   2.89     0  0.445  6.163  69.6  3.4952    2  276     18.0   
97  0.12083   0.0   2.89     0  0.445  8.069  76.0  3.4952    2  276     18.0   
98  0.08187   0.0   2.89     0  0.445  7.820  36.9  3.4952    2  276     18.0   
99  0.06860   0.0   2.89     0  0.445  7.416  62.5  3.4952    2  276     18.0   

         B  LSTA